In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

# Occasions Annotations

## Bin of 6+ `guest_count`
Let's look at a few tables:
- `hockey_3_text_processed.csv`
- `silvester_3_text_processed.csv`
- `valentine_3_text_processed.csv`

to get an idea of the different occasions.

###### Our initial set of **occasions** for the category 6+ bin is: 
1. `Drinking` - purpose to drink
    - `Social drinking`
    - `Party`
    
    
2. `Meal` - purpose is to eat
    - `Dinner`
    - `Lunch`
    
    
3. `Social gathring` 
    - `after work` 
    
    
4. `Family Occasion` 

    


### Features:
1. time of day
2. food to drinks ratio
3. kid meal
4. meal step



----------

In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 50)

In [5]:
hockey_path = '../data/hockey_3_text_processed.csv'
silvester_path = '../data/silvester_3_text_processed.csv'
valentine_path = '../data/valentine_3_text_processed.csv'

In [6]:
hockey = pd.read_csv(hockey_path)
silvester = pd.read_csv(silvester_path)
valentine = pd.read_csv(valentine_path)

Fix `datetime` features:

In [7]:
def fix_times(df):
    df['order_time'] = pd.to_datetime(df['order_time'], format="%Y-%m-%d %H:%M:%S.%f")
    df['order_time_closed'] = pd.to_datetime(df['order_time_closed'], format="%Y-%m-%d %H:%M:%S.%f")
    df['order_item_time'] = pd.to_datetime(df['order_item_time'], format="%Y-%m-%d %H:%M:%S.%f")

In [8]:
fix_times(hockey)
fix_times(valentine)
fix_times(silvester)

Lets only look at interesting features

In [9]:
features = ['order_id', 'order_time', 'order_time_closed',
       'order_item_time', 'title', 'category_id', 
       'dwell_time', 'meal_step',
       'meal_flow_last_to_close', 'total_flow_steps', 'meal_flow_step', 
       'beer_volume', 'order_day_of_week',
       'is_weekend', 'guest_count',
       'period_of_day', 
       'sharable', 'kids_meal', 'birthday',
       'total_sales_before_tax', 'item_qty', 'sales_before_tax']

In [10]:
hockey = hockey[features]
silvester = silvester[features]
valentine = valentine[features]

In [11]:
hockey.head()

,order_id,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
0,520008106,2019-04-16 12:01:00,2019-04-16 13:15:52,2019-04-16 12:13:26,$Sub.Swt.Pot.Fry-,2.0,1.247778,1,15,18,3,0.000,Tuesday,False,1,lunch,0,0,0,19.84,1.0,2.97
1,520008106,2019-04-16 12:01:00,2019-04-16 13:15:52,2019-04-16 12:13:26,FINGERS.SAUCED,2.0,1.247778,1,15,18,3,0.000,Tuesday,False,1,lunch,1,0,0,19.84,1.0,16.87
2,520008105,2019-04-16 12:01:00,2019-04-16 13:11:55,2019-04-16 12:18:25,TRAD.GUAC,2.0,1.181944,2,13,17,4,0.000,Tuesday,False,1,lunch,1,0,0,26.61,1.0,8.99
3,520008105,2019-04-16 12:01:00,2019-04-16 13:11:55,2019-04-16 12:13:26,VEGGIE.BURGER,2.0,1.181944,1,13,17,3,0.000,Tuesday,False,1,lunch,0,0,0,26.61,1.0,17.62
4,520253565,2019-04-16 19:35:00,2019-04-16 22:39:18,2019-04-16 20:56:13,CANADIAN.Pt,1.0,3.071667,2,12,44,19,0.473,Tuesday,False,0,dinner,0,0,0,30.48,1.0,7.62


Create table for each order, we're going to pick (randomly) 10 tables out of each dataframe.

In [11]:
def create_tables(df):
    tables = df.drop_duplicates(subset='order_id', keep='first')
    tables.set_index('order_id', inplace=True)
    tables = tables[['guest_count']]

    return tables

In [12]:
hockey_tables = create_tables(hockey)
valentine_tables = create_tables(valentine)
silvester_tables = create_tables(silvester)

In [12]:
print("We have ", len(hockey_tables), " tables in the Hockey data")

We have  118664  tables in the Hockey data


In [13]:
print("We have ", len(valentine_tables), " tables in the Valentine data")

We have  40787  tables in the Valentine data


In [14]:
print("We have ", len(silvester_tables), " tables in the Silvester data")

We have  92107  tables in the Silvester data


In [13]:
hockey.set_index('order_id', inplace=True)
valentine.set_index('order_id', inplace=True)
silvester.set_index('order_id', inplace=True)

-----

# The Annotations

Pick 10 tables out of each dataframe.

In [16]:
COUNT = 10

In [19]:
import random

In [14]:
def get_tables_gc_6(tables):
    return tables[tables['guest_count'] >5]

## Annotating Hockey

Pick 10 tables of a total of 1 guests of **Hockey**.

In [18]:
hockey_6gc_tables = get_tables_gc_6(hockey_tables)
print("We have {} tables of a total of 6+ guests".format(len(hockey_6gc_tables)))

We have 2377 tables of a total of 6+ guests


In [22]:
picked_hockey_tables = random.sample(list(hockey_6gc_tables.index), COUNT)

In [19]:
picked_hockey_tables = [514594544,
 520144566,
 512856753,
 523552965,
 522777307,
 520932269,
 520961529,
 522822209,
 524870243,
 514742701]

# Table 1

In [20]:
#514534188
hockey[hockey.index == picked_hockey_tables[0]].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
514594544,2019-04-13 19:58:39,2019-04-13 21:23:25,2019-04-13 20:04:17,Tex Steak Sand,2.0,1.412778,1,6,19,1,0.000,Saturday,True,10,dinner,0,0,0,199.603372,1.0,15.762830
514594544,2019-04-13 19:58:39,2019-04-13 21:23:25,2019-04-13 20:04:17,P.B Deli Sand,2.0,1.412778,1,6,19,1,0.000,Saturday,True,10,dinner,0,0,0,199.603372,1.0,12.805451
514594544,2019-04-13 19:58:39,2019-04-13 21:23:25,2019-04-13 20:04:17,Onion Soup,2.0,1.412778,1,6,19,1,0.000,Saturday,True,10,dinner,0,0,0,199.603372,1.0,5.904900
514594544,2019-04-13 19:58:39,2019-04-13 21:23:25,2019-04-13 20:04:17,Fish & Chips,2.0,1.412778,1,6,19,1,0.000,Saturday,True,10,dinner,0,0,0,199.603372,1.0,13.791244
514594544,2019-04-13 19:58:39,2019-04-13 21:23:25,2019-04-13 20:04:17,Fount Pop,4.0,1.412778,1,6,19,1,0.000,Saturday,True,10,dinner,0,0,0,199.603372,1.0,2.947521
514594544,2019-04-13 19:58:39,2019-04-13 21:23:25,2019-04-13 20:04:17,Fish & Chips,2.0,1.412778,1,6,19,1,0.000,Saturday,True,10,dinner,0,0,0,199.603372,1.0,13.791244
514594544,2019-04-13 19:58:39,2019-04-13 21:23:25,2019-04-13 20:04:17,Tex Steak Sand,2.0,1.412778,1,6,19,1,0.000,Saturday,True,10,dinner,0,0,0,199.603372,1.0,15.762830
514594544,2019-04-13 19:58:39,2019-04-13 21:23:25,2019-04-13 20:04:17,Hot Beef Sand,2.0,1.412778,1,6,19,1,0.000,Saturday,True,10,dinner,0,0,0,199.603372,1.0,14.777037
514594544,2019-04-13 19:58:39,2019-04-13 21:23:25,2019-04-13 20:04:17,Chicken Fingers,2.0,1.412778,1,6,19,1,0.000,Saturday,True,10,dinner,1,0,0,199.603372,1.0,19.686287


# **April 13th (Saturday), 19:58 - 21:23, a total of 1.5 hours**

**Meal Steps Description:**
1. Start
  - Meal very heavy, all ordered at once

2. Coffe 30 min later
  - Dessert

      
**`Dinner`** 


**Explanation**: Because of the time and the size of the meal.

**Features that helped**: the amount of foold on the table is approxemtly the GC, `price of meal` (10 $), and `time of meal` (21:00pm). 

# Table 2

In [21]:
hockey[hockey.index == picked_hockey_tables[1]].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
520144566,2019-04-16 19:51:09,2019-04-16 20:37:33,2019-04-16 19:51:43,Deluxe Dinner,2.0,0.773333,1,0,10,0,-0.000,Tuesday,False,22,dinner,0,0,0,3253.84,-1.0,0.000000
520144566,2019-04-16 19:51:09,2019-04-16 20:37:33,2019-04-16 19:51:43,Deluxe Dinner,2.0,0.773333,1,0,10,0,0.000,Tuesday,False,22,dinner,0,0,0,3253.84,22.0,2259.481435
520144566,2019-04-16 19:51:09,2019-04-16 20:37:33,2019-04-16 19:51:43,Deluxe Dinner,2.0,0.773333,1,0,10,0,-0.000,Tuesday,False,22,dinner,0,0,0,3253.84,-1.0,0.000000
520144566,2019-04-16 19:51:09,2019-04-16 20:37:33,2019-04-16 20:00:32,Pop,4.0,0.773333,2,0,10,2,0.000,Tuesday,False,22,dinner,0,0,0,3253.84,2.0,5.835438
520144566,2019-04-16 19:51:09,2019-04-16 20:37:33,2019-04-16 20:00:32,Evian,4.0,0.773333,2,0,10,2,0.000,Tuesday,False,22,dinner,0,0,0,3253.84,9.0,91.908142
520144566,2019-04-16 19:51:09,2019-04-16 20:37:33,2019-04-16 20:00:32,Guinness,1.0,0.773333,2,0,10,2,0.440,Tuesday,False,22,dinner,0,0,0,3253.84,1.0,9.336700
520144566,2019-04-16 19:51:09,2019-04-16 20:37:33,2019-04-16 20:00:32,Heineken,1.0,0.773333,2,0,10,2,0.660,Tuesday,False,22,dinner,0,0,0,3253.84,2.0,18.673400
520144566,2019-04-16 19:51:09,2019-04-16 20:37:33,2019-04-16 20:00:32,Castle Cab Sauv,6.0,0.773333,2,0,10,2,0.000,Tuesday,False,22,dinner,0,0,0,3253.84,6.0,406.146456
520144566,2019-04-16 19:51:09,2019-04-16 20:37:33,2019-04-16 20:00:32,San Pellegrino,4.0,0.773333,2,0,10,2,0.000,Tuesday,False,22,dinner,0,0,0,3253.84,7.0,71.484110


**April 16th (Tuesday), 19:51 - 20:37, total 0.77 hours, 2 meal steps**

** 22 people eat dinner and some order more soft drinks and liquer 
I dont use the the dwell time here
**Meal Steps Description:**
it actually has a single meal flow and not 2 as mentioned. 
       
**Total drinks**: 5 alcholic drinks 3 soft drinks

**Dinner Party**

**Explanation**: This is a party where the foucus is on the food, room was set for the group. 

**Features**: number of dinners which was served for the table, amount of food over the amount of drinks

rule: if the guest count is greather than 12 then it was set before, therefore it a big event and with high ratio of food over drinks it is a dinner event. 

# Table 3

In [28]:
#521093892
hockey[hockey.index == picked_hockey_tables[2]].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
512856753,2019-04-11 20:35:00,2019-04-12 01:25:07,2019-04-11 20:35:00,LG COORS LIGHT,1.0,4.835278,1,8,70,0,0.591,Thursday,False,11,dinner,0,0,0,76.75,1.0,7.25
512856753,2019-04-11 20:35:00,2019-04-12 01:25:07,2019-04-11 20:41:00,CHICKEN STIR FRY,2.0,4.835278,2,8,70,1,0.000,Thursday,False,11,dinner,0,0,0,76.75,1.0,16.00
512856753,2019-04-11 20:35:00,2019-04-12 01:25:07,2019-04-11 21:21:00,LG COORS LIGHT,1.0,4.835278,2,8,70,11,0.591,Thursday,False,11,dinner,0,0,0,76.75,1.0,7.25
512856753,2019-04-11 20:35:00,2019-04-12 01:25:07,2019-04-11 22:31:00,LG COORS LIGHT,1.0,4.835278,2,8,70,28,0.591,Thursday,False,11,dinner,0,0,0,76.75,1.0,7.25
512856753,2019-04-11 20:35:00,2019-04-12 01:25:07,2019-04-11 23:27:00,LG COORS LIGHT,1.0,4.835278,2,8,70,42,0.591,Thursday,False,11,dinner,0,0,0,76.75,1.0,7.25
512856753,2019-04-11 20:35:00,2019-04-12 01:25:07,2019-04-12 00:28:00,LG COORS LIGHT,1.0,4.835278,2,8,70,57,0.591,Thursday,False,11,dinner,0,0,0,76.75,1.0,7.25
512856753,2019-04-11 20:35:00,2019-04-12 01:25:07,2019-04-12 00:37:00,LG COORS LIGHT,1.0,4.835278,2,8,70,59,0.591,Thursday,False,11,dinner,0,0,0,76.75,1.0,7.25
512856753,2019-04-11 20:35:00,2019-04-12 01:25:07,2019-04-12 00:37:00,LG COORS LIGHT,1.0,4.835278,2,8,70,59,0.591,Thursday,False,11,dinner,0,0,0,76.75,1.0,7.25
512856753,2019-04-11 20:35:00,2019-04-12 01:25:07,2019-04-12 00:52:00,PEPPERONI SML,2.0,4.835278,2,8,70,62,0.000,Thursday,False,11,dinner,0,0,0,76.75,1.0,10.00


**Social Gathering**

**April 11th (Thursday), 20:35 - 01:25, total of 4.83 hours**

**Meal Steps Description:**
1. food and beer
2. 40 minutes later: beer
3. 70 minutes later: beer
4. 60 min later: beer
5. 60 min later: 2 beer
6. 15 min later: pizza


**`Wrong Label`**


**Explanation**: Its hard to say how many people are on the table but it is less than 11 and probabily less than 6 so I classifity this as a wornk label of GC



# Table 4

In [29]:
hockey[hockey.index == picked_hockey_tables[3]].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
523552965,2019-04-20 18:42:00,2019-04-20 20:31:50,2019-04-20 18:42:00,SODA,4.0,1.830556,1,7,26,0,0.00,Saturday,True,6,dinner,0,0,0,188.77,1.0,3.00
523552965,2019-04-20 18:42:00,2019-04-20 20:31:50,2019-04-20 18:42:00,COKE,4.0,1.830556,1,7,26,0,0.00,Saturday,True,6,dinner,0,0,0,188.77,1.0,3.00
523552965,2019-04-20 18:42:00,2019-04-20 20:31:50,2019-04-20 18:50:00,CALAMARI,2.0,1.830556,2,7,26,2,0.00,Saturday,True,6,dinner,1,0,0,188.77,1.0,13.67
523552965,2019-04-20 18:42:00,2019-04-20 20:31:50,2019-04-20 18:50:00,CALAMARI,2.0,1.830556,2,7,26,2,0.00,Saturday,True,6,dinner,1,0,0,188.77,1.0,13.67
523552965,2019-04-20 18:42:00,2019-04-20 20:31:50,2019-04-20 19:04:00,CHICKN FAJITA,2.0,1.830556,2,7,26,5,0.00,Saturday,True,6,dinner,0,0,0,188.77,1.0,21.88
523552965,2019-04-20 18:42:00,2019-04-20 20:31:50,2019-04-20 19:04:00,HALIBUT&CHIPS,2.0,1.830556,2,7,26,5,0.00,Saturday,True,6,dinner,0,0,0,188.77,1.0,24.88
523552965,2019-04-20 18:42:00,2019-04-20 20:31:50,2019-04-20 19:04:00,STEAK FAJITA,2.0,1.830556,2,7,26,5,0.00,Saturday,True,6,dinner,0,0,0,188.77,1.0,22.43
523552965,2019-04-20 18:42:00,2019-04-20 20:31:50,2019-04-20 19:04:00,STEAK FAJITA,2.0,1.830556,2,7,26,5,0.00,Saturday,True,6,dinner,0,0,0,188.77,1.0,22.43
523552965,2019-04-20 18:42:00,2019-04-20 20:31:50,2019-04-20 19:04:00,2LB WINGS,2.0,1.830556,2,7,26,5,0.00,Saturday,True,6,dinner,1,0,0,188.77,1.0,25.92


**`Dinner`**
Family dinner 


There is also a "kid" meal in here.

many foods and single beer with two soft drinks

**Features**: GC size of a family, rator of food to alcohol beverage is high. small number of soft drinks

# Table 5

In [30]:
hockey[hockey.index == picked_hockey_tables[4]].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
522777307,2019-04-20 00:06:00,2019-04-20 00:45:46,2019-04-20 00:06:32,JACK DANIELS DBL,3.0,0.662778,1,2,8,0,0.000,Saturday,True,6,night,0,0,0,132.45,1.0,10.50
522777307,2019-04-20 00:06:00,2019-04-20 00:45:46,2019-04-20 00:06:32,CUERVO GOLD,3.0,0.662778,1,2,8,0,0.000,Saturday,True,6,night,0,0,0,132.45,1.0,6.50
522777307,2019-04-20 00:06:00,2019-04-20 00:45:46,2019-04-20 00:06:32,CORONA,1.0,0.662778,1,2,8,0,0.355,Saturday,True,6,night,0,0,0,132.45,1.0,7.50
522777307,2019-04-20 00:06:00,2019-04-20 00:45:46,2019-04-20 00:06:32,CORONA,1.0,0.662778,1,2,8,0,0.355,Saturday,True,6,night,0,0,0,132.45,1.0,7.50
522777307,2019-04-20 00:06:00,2019-04-20 00:45:46,2019-04-20 00:06:32,RED BULL,4.0,0.662778,1,2,8,0,0.000,Saturday,True,6,night,0,0,0,132.45,1.0,4.25
522777307,2019-04-20 00:06:00,2019-04-20 00:45:46,2019-04-20 00:06:32,HENNESSY DBL,3.0,0.662778,1,2,8,0,0.000,Saturday,True,6,night,0,0,0,132.45,1.0,26.50
522777307,2019-04-20 00:06:00,2019-04-20 00:45:46,2019-04-20 00:24:55,BURT REYNOLDS,3.0,0.662778,2,2,8,4,0.000,Saturday,True,6,night,0,0,0,132.45,1.0,4.95
522777307,2019-04-20 00:06:00,2019-04-20 00:45:46,2019-04-20 00:24:55,CORONA,1.0,0.662778,2,2,8,4,0.355,Saturday,True,6,night,0,0,0,132.45,1.0,7.50
522777307,2019-04-20 00:06:00,2019-04-20 00:45:46,2019-04-20 00:24:55,HENNESSY DBL,3.0,0.662778,2,2,8,4,0.000,Saturday,True,6,night,0,0,0,132.45,1.0,26.50


**April 20th (Sauturday), 00:06 - 00:45, total of 0.66 hours**

**`drinking`**

only drinks on the table, 2 meal step and with six rows per meal step. 


**Explanation**: Not much to explain, a group of people are drinking.


# Table 6

In [31]:
hockey[hockey.index == picked_hockey_tables[5]].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
520932269,2019-04-17 17:20:00,2019-04-17 18:43:31,2019-04-17 17:20:51,POP.,4.0,1.391944,1,8,16,0,0.000,Wednesday,False,6,afternoon,0,0,0,111.59,1.0,3.99
520932269,2019-04-17 17:20:00,2019-04-17 18:43:31,2019-04-17 17:20:51,CREEMORE.Pt,1.0,1.391944,1,8,16,0,0.473,Wednesday,False,6,afternoon,0,0,0,111.59,1.0,8.72
520932269,2019-04-17 17:20:00,2019-04-17 18:43:31,2019-04-17 17:28:38,POP.,4.0,1.391944,2,8,16,1,0.000,Wednesday,False,6,afternoon,0,0,0,111.59,1.0,3.99
520932269,2019-04-17 17:20:00,2019-04-17 18:43:31,2019-04-17 17:28:38,POP.,4.0,1.391944,2,8,16,1,0.000,Wednesday,False,6,afternoon,0,0,0,111.59,1.0,3.99
520932269,2019-04-17 17:20:00,2019-04-17 18:43:31,2019-04-17 17:29:39,Loaded.Mash,2.0,1.391944,1,8,16,1,0.000,Wednesday,False,6,afternoon,0,0,0,111.59,1.0,0.99
520932269,2019-04-17 17:20:00,2019-04-17 18:43:31,2019-04-17 17:30:17,SAM.ADAM.SEAS.Pt,1.0,1.391944,1,8,16,1,0.597,Wednesday,False,6,afternoon,0,0,0,111.59,1.0,9.33
520932269,2019-04-17 17:20:00,2019-04-17 18:43:31,2019-04-17 17:30:17,BELGIAN.MOON..Pt,1.0,1.391944,1,8,16,1,0.473,Wednesday,False,6,afternoon,0,0,0,111.59,1.0,8.43
520932269,2019-04-17 17:20:00,2019-04-17 18:43:31,2019-04-17 17:33:39,TUNA.GUAC,2.0,1.391944,1,8,16,1,0.000,Wednesday,False,6,afternoon,1,0,0,111.59,1.0,14.97
520932269,2019-04-17 17:20:00,2019-04-17 18:43:31,2019-04-17 17:41:33,FINGERS.SAUCED,2.0,1.391944,2,8,16,2,0.000,Wednesday,False,6,afternoon,1,0,0,111.59,1.0,16.87


**April 17th (Wednesday), 17:20 - 18:43, total of 1.39 hours,.**

**`Dinner`**

**Explanation**: different types of food and varity of drinks, seems like a grown up group gathering after work.
**Features**:  day of week, hour, ration of food to drinks is almost 1



# Table 7

In [29]:
hockey[hockey.index == picked_hockey_tables[6]].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
520961529,2019-04-17 18:22:00,2019-04-17 20:00:21,2019-04-17 18:22:01,RICKARD.RED.Pt,1.0,1.639167,1,8,21,0,0.473,Wednesday,False,10,dinner,0,0,0,214.57,1.0,8.43
520961529,2019-04-17 18:22:00,2019-04-17 20:00:21,2019-04-17 18:22:01,HEINEKEN.Pt,1.0,1.639167,1,8,21,0,0.562,Wednesday,False,10,dinner,0,0,0,214.57,1.0,9.33
520961529,2019-04-17 18:22:00,2019-04-17 20:00:21,2019-04-17 18:22:01,KIDBEV,4.0,1.639167,1,8,21,0,0.000,Wednesday,False,10,dinner,0,0,0,214.57,1.0,0.00
520961529,2019-04-17 18:22:00,2019-04-17 20:00:21,2019-04-17 18:22:01,KIDBEV,4.0,1.639167,1,8,21,0,0.000,Wednesday,False,10,dinner,0,0,0,214.57,1.0,0.00
520961529,2019-04-17 18:22:00,2019-04-17 20:00:21,2019-04-17 18:22:01,KIDBEV,4.0,1.639167,1,8,21,0,0.000,Wednesday,False,10,dinner,0,0,0,214.57,1.0,0.00
520961529,2019-04-17 18:22:00,2019-04-17 20:00:21,2019-04-17 18:22:01,POP.,4.0,1.639167,1,8,21,0,0.000,Wednesday,False,10,dinner,0,0,0,214.57,1.0,3.99
520961529,2019-04-17 18:22:00,2019-04-17 20:00:21,2019-04-17 18:22:27,KIDBEV,4.0,1.639167,1,8,21,0,0.000,Wednesday,False,10,dinner,0,0,0,214.57,1.0,0.00
520961529,2019-04-17 18:22:00,2019-04-17 20:00:21,2019-04-17 18:22:27,WATER.,4.0,1.639167,1,8,21,0,0.000,Wednesday,False,10,dinner,0,0,0,214.57,1.0,0.00
520961529,2019-04-17 18:22:00,2019-04-17 20:00:21,2019-04-17 18:27:34,RICKARD.RED.Pt,1.0,1.639167,2,8,21,1,0.473,Wednesday,False,10,dinner,0,0,0,214.57,1.0,8.43


**April 17th (Wednesday), 18:22 - 20:00, total of 1.63 hours**

**`Family`**        

**Explanation**: Its a classic dinner, maybe also watchign a game or something. Starts with a drinks and order food for many people.later order some desserts.


**Features**: staying at the table for 2 hours, many drinks and many food, ratio is almost 0.5  

# Table 8

In [28]:
hockey[hockey.index == picked_hockey_tables[7]].sort_values(by=['order_item_time'])

NameError: name 'picked_hockey_tables' is not defined

**April 20nd (Saturday), 01:48 - 02:18, total of 0.5 hours**

       
**total drinks**: 4 beers and 10 liquers,

**`Social drinking`**

**Explanation**: Its a drinking table, Its a late night, they came to drink.

**Features**: staying at the table for 30 minutes, food to drink ratio is 0. many drinks, hard liquer

# Valentine

Pick 5 tables of total of 1 guests for **Valentine**.

In [24]:
valentine_gc = get_tables_gc_6(valentine_tables)

print("We have {} tables of a total of 1 guests".format(len(valentine_gc)))

We have 720 tables of a total of 1 guests


In [26]:
import random
picked_valentine_tables = random.sample(list(valentine_gc.index), COUNT)

In [28]:
picked_valentine_tables = [434693771,
 446750517,
 447075894,
 447589967,
 434539751,
 448045237,
 448730983,
 447983682,
 447100562,
 434734939]

## Table 1

In [29]:
valentine[valentine.index == picked_valentine_tables[0]].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
434693771,2019-02-10 17:38:00,2019-02-10 19:46:28,2019-02-10 17:38:00,W.KIM.CRAW.SV.1z,6.0,2.141111,1,19,31,0,0.0,Sunday,True,7,afternoon,0,0,0,380.5,1.0,21.00
434693771,2019-02-10 17:38:00,2019-02-10 19:46:28,2019-02-10 17:51:00,R.ST.JEAN.CAB.Bt,6.0,2.141111,2,19,31,3,0.0,Sunday,True,7,afternoon,0,0,0,380.5,1.0,55.00
434693771,2019-02-10 17:38:00,2019-02-10 19:46:28,2019-02-10 18:04:00,7OZ.SIRLOIN,2.0,2.141111,3,19,31,6,0.0,Sunday,True,7,afternoon,0,0,0,380.5,1.0,29.45
434693771,2019-02-10 17:38:00,2019-02-10 19:46:28,2019-02-10 18:04:00,BACON.FILET*,2.0,2.141111,3,19,31,6,0.0,Sunday,True,7,afternoon,0,0,0,380.5,1.0,40.95
434693771,2019-02-10 17:38:00,2019-02-10 19:46:28,2019-02-10 18:04:00,LAMB.SALAD*,2.0,2.141111,3,19,31,6,0.0,Sunday,True,7,afternoon,0,0,0,380.5,1.0,21.75
434693771,2019-02-10 17:38:00,2019-02-10 19:46:28,2019-02-10 18:04:00,$Truffle.Fries$$,2.0,2.141111,3,19,31,6,0.0,Sunday,True,7,afternoon,1,0,0,380.5,1.0,3.95
434693771,2019-02-10 17:38:00,2019-02-10 19:46:28,2019-02-10 18:04:00,CAESAR.APP,2.0,2.141111,3,19,31,6,0.0,Sunday,True,7,afternoon,0,0,0,380.5,1.0,8.95
434693771,2019-02-10 17:38:00,2019-02-10 19:46:28,2019-02-10 18:04:00,TUNA.STACK,2.0,2.141111,3,19,31,6,0.0,Sunday,True,7,afternoon,0,0,0,380.5,1.0,15.50
434693771,2019-02-10 17:38:00,2019-02-10 19:46:28,2019-02-10 18:04:00,b.potato,2.0,2.141111,3,19,31,6,0.0,Sunday,True,7,afternoon,0,0,0,380.5,1.0,0.00


**Feb 10th (Sunday), 17:38 - 19:46, total of ~2 hour, 5 meal steps.**

**total drinks**: 3 wines,

**`family dinner`**


# Table 2

In [30]:
valentine[valentine.index == picked_valentine_tables[1]].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
446750517,2019-02-12 12:07:00,2019-02-12 13:09:18,2019-02-12 12:07:00,LRG BELGIAN MOON,1.0,1.038333,1,6,14,0,0.591,Tuesday,False,8,lunch,0,0,0,57.85,1.0,7.25
446750517,2019-02-12 12:07:00,2019-02-12 13:09:18,2019-02-12 12:07:00,HEINEKEN,1.0,1.038333,1,6,14,0,0.330,Tuesday,False,8,lunch,0,0,0,57.85,1.0,7.20
446750517,2019-02-12 12:07:00,2019-02-12 13:09:18,2019-02-12 12:18:00,CHEF SPECIAL,2.0,1.038333,2,6,14,2,0.000,Tuesday,False,8,lunch,0,0,0,57.85,1.0,16.97
446750517,2019-02-12 12:07:00,2019-02-12 13:09:18,2019-02-12 12:18:00,CHKN CLUBHOUSE,2.0,1.038333,2,6,14,2,0.000,Tuesday,False,8,lunch,0,0,0,57.85,1.0,15.00
446750517,2019-02-12 12:07:00,2019-02-12 13:09:18,2019-02-12 12:32:00,HEINEKEN,1.0,1.038333,3,6,14,5,0.330,Tuesday,False,8,lunch,0,0,0,57.85,1.0,7.20
446750517,2019-02-12 12:07:00,2019-02-12 13:09:18,2019-02-12 12:45:00,RG SLEEMAN,1.0,1.038333,4,6,14,8,0.355,Tuesday,False,8,lunch,0,0,0,57.85,1.0,4.23


**Feb 12th (Tuesday), 12:07 - 13:09, total of 1 hour , 4 meal steps.**


       
**total meals**: 2 foods 4 beers

wrong label






# Table 3

In [31]:
valentine[valentine.index == picked_valentine_tables[2]].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
447075894,2019-02-13 18:41:00,2019-02-13 19:52:42,2019-02-13 18:41:00,POP.,4.0,1.195,1,12,16,0,0.000,Wednesday,False,9,dinner,0,0,0,232.12,1.0,3.99
447075894,2019-02-13 18:41:00,2019-02-13 19:52:42,2019-02-13 18:41:00,BLURASP LEMONADE,3.0,1.195,1,12,16,0,0.000,Wednesday,False,9,dinner,0,0,0,232.12,1.0,8.97
447075894,2019-02-13 18:41:00,2019-02-13 19:52:42,2019-02-13 18:41:00,BAR.MANGER.XL,1.0,1.195,1,12,16,0,0.591,Wednesday,False,9,dinner,0,0,0,232.12,1.0,10.30
447075894,2019-02-13 18:41:00,2019-02-13 19:52:42,2019-02-13 18:41:00,DBL.BARRL.SHZ.Lg,6.0,1.195,1,12,16,0,0.000,Wednesday,False,9,dinner,0,0,0,232.12,1.0,17.98
447075894,2019-02-13 18:41:00,2019-02-13 19:52:42,2019-02-13 18:41:00,BLURASP LEMONADE,3.0,1.195,1,12,16,0,0.000,Wednesday,False,9,dinner,0,0,0,232.12,1.0,8.97
447075894,2019-02-13 18:41:00,2019-02-13 19:52:42,2019-02-13 18:41:00,MARGRITA.LIME,3.0,1.195,1,12,16,0,0.000,Wednesday,False,9,dinner,0,0,0,232.12,1.0,8.97
447075894,2019-02-13 18:41:00,2019-02-13 19:52:42,2019-02-13 18:41:00,SML.RED.SANG,3.0,1.195,1,12,16,0,0.000,Wednesday,False,9,dinner,0,0,0,232.12,1.0,13.29
447075894,2019-02-13 18:41:00,2019-02-13 19:52:42,2019-02-13 18:41:00,STELLA.Pt,1.0,1.195,1,12,16,0,0.591,Wednesday,False,9,dinner,0,0,0,232.12,1.0,9.33
447075894,2019-02-13 18:41:00,2019-02-13 19:52:42,2019-02-13 18:41:00,BEACHED,3.0,1.195,1,12,16,0,0.000,Wednesday,False,9,dinner,0,0,0,232.12,1.0,8.97


**Feb 13th (Wednesday), 13:08 - 14:07, total of ~1 hour, 3 meal steps.**

**total drinks**: 10 
**total food**: 9

**`Dinner`**

**Explanation**: classic Dinner. 

**Features**: time of meal , and spent about an hour.

# Table 4

In [32]:
valentine[valentine.index == picked_valentine_tables[3]].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
447589967,2019-02-14 17:46:00,2019-02-14 20:59:33,2019-02-14 17:46:00,STELLA,1.0,3.225833,1,46,48,0,0.330,Thursday,False,6,afternoon,0,0,0,84.15,1.0,7.15
447589967,2019-02-14 17:46:00,2019-02-14 20:59:33,2019-02-14 17:46:00,LRG RICKARDS,1.0,3.225833,1,46,48,0,0.591,Thursday,False,6,afternoon,0,0,0,84.15,1.0,7.75
447589967,2019-02-14 17:46:00,2019-02-14 20:59:33,2019-02-14 17:46:00,LRG BELGIAN MOON,1.0,3.225833,1,46,48,0,0.591,Thursday,False,6,afternoon,0,0,0,84.15,1.0,7.25
447589967,2019-02-14 17:46:00,2019-02-14 20:59:33,2019-02-14 17:46:00,1/2 LITRE RED,6.0,3.225833,1,46,48,0,0.000,Thursday,False,6,afternoon,0,0,0,84.15,1.0,18.00
447589967,2019-02-14 17:46:00,2019-02-14 20:59:33,2019-02-14 17:55:00,STEAK BITES,2.0,3.225833,2,46,48,2,0.000,Thursday,False,6,afternoon,0,0,0,84.15,1.0,8.00
447589967,2019-02-14 17:46:00,2019-02-14 20:59:33,2019-02-14 17:55:00,EIGHT WINGS,2.0,3.225833,2,46,48,2,0.000,Thursday,False,6,afternoon,1,0,0,84.15,1.0,14.00
447589967,2019-02-14 17:46:00,2019-02-14 20:59:33,2019-02-14 17:55:00,1.50,2.0,3.225833,2,46,48,2,0.000,Thursday,False,6,afternoon,0,0,0,84.15,1.0,1.50
447589967,2019-02-14 17:46:00,2019-02-14 20:59:33,2019-02-14 17:55:00,CAULIFLOWER BITES,2.0,3.225833,2,46,48,2,0.000,Thursday,False,6,afternoon,0,0,0,84.15,1.0,6.00
447589967,2019-02-14 17:46:00,2019-02-14 20:59:33,2019-02-14 17:55:00,SAUSAGE MARGESE,2.0,3.225833,2,46,48,2,0.000,Thursday,False,6,afternoon,0,0,0,84.15,1.0,6.50


**`dinner`**

food to drinks ration is higer than 1

time of day

total amout of orders

# Silvester

In [18]:
import random
silvester_gc = get_tables_gc_6(silvester_tables)
print("We have {} tables of a total of 1 guests".format(len(silvester_gc)))

We have 2190 tables of a total of 1 guests


In [20]:
picked_silvester_tables = random.sample(list(silvester_gc.index), 3)

In [22]:
picked_silvester_tables = [361348245, 360970732, 360903046]

# Table 1

In [33]:
silvester[silvester.index == picked_silvester_tables[0]].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
361348245,2018-12-29 19:05:31,2018-12-29 19:55:18,2018-12-29 19:09:13,Fettuccine Pomo,2.0,0.829722,1,11,11,0,0.0,Saturday,True,9,dinner,0,0,0,158.89,1.0,16.99
361348245,2018-12-29 19:05:31,2018-12-29 19:55:18,2018-12-29 19:09:13,3 Alarm Burger,2.0,0.829722,1,11,11,0,0.0,Saturday,True,9,dinner,0,0,0,158.89,1.0,14.99
361348245,2018-12-29 19:05:31,2018-12-29 19:55:18,2018-12-29 19:09:13,PB House Burger,2.0,0.829722,1,11,11,0,0.0,Saturday,True,9,dinner,0,0,0,158.89,1.0,14.99
361348245,2018-12-29 19:05:31,2018-12-29 19:55:18,2018-12-29 19:09:13,Great Reuben,5.0,0.829722,1,11,11,0,0.0,Saturday,True,9,dinner,0,0,0,158.89,1.0,14.99
361348245,2018-12-29 19:05:31,2018-12-29 19:55:18,2018-12-29 19:09:13,Gnocchi Tomato,2.0,0.829722,1,11,11,0,0.0,Saturday,True,9,dinner,0,0,0,158.89,1.0,16.99
361348245,2018-12-29 19:05:31,2018-12-29 19:55:18,2018-12-29 19:09:13,S-Taco Baja Fish,2.0,0.829722,1,11,11,0,0.0,Saturday,True,9,dinner,0,0,0,158.89,1.0,5.99
361348245,2018-12-29 19:05:31,2018-12-29 19:55:18,2018-12-29 19:09:13,Mexican Shrimp,2.0,0.829722,1,11,11,0,0.0,Saturday,True,9,dinner,0,0,0,158.89,1.0,16.99
361348245,2018-12-29 19:05:31,2018-12-29 19:55:18,2018-12-29 19:09:13,Matzo Ball Soup,2.0,0.829722,1,11,11,0,0.0,Saturday,True,9,dinner,0,0,0,158.89,1.0,4.99
361348245,2018-12-29 19:05:31,2018-12-29 19:55:18,2018-12-29 19:09:13,P.B Deli Sand,2.0,0.829722,1,11,11,0,0.0,Saturday,True,9,dinner,0,0,0,158.89,1.0,12.99


**DEC 29th (Saturday), 19:05 - 19:55, total of 50 minutes, 1 meal steps.**

**`dinner`** 

`dinner_time`, `dewll_meal`


# Table 2

In [25]:
silvester[silvester.index == picked_silvester_tables[1]].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
360970732,2018-12-28 21:39:51,2018-12-28 23:01:16,2018-12-28 21:40:10,Green Monster,4.0,1.356944,1,6,18,0,0.0,Friday,True,6,hang_out,0,0,0,130.87,1.0,5.49
360970732,2018-12-28 21:39:51,2018-12-28 23:01:16,2018-12-28 21:40:10,Killer Shake,4.0,1.356944,1,6,18,0,0.0,Friday,True,6,hang_out,0,0,0,130.87,1.0,4.99
360970732,2018-12-28 21:39:51,2018-12-28 23:01:16,2018-12-28 21:49:23,Maple Nut Salmon,2.0,1.356944,2,6,18,2,0.0,Friday,True,6,hang_out,0,0,0,130.87,1.0,18.99
360970732,2018-12-28 21:39:51,2018-12-28 23:01:16,2018-12-28 21:49:23,Fettuccine Pomo,2.0,1.356944,2,6,18,2,0.0,Friday,True,6,hang_out,0,0,0,130.87,1.0,18.48
360970732,2018-12-28 21:39:51,2018-12-28 23:01:16,2018-12-28 21:49:23,Spag & Meatballs,2.0,1.356944,2,6,18,2,0.0,Friday,True,6,hang_out,0,0,0,130.87,1.0,15.99
360970732,2018-12-28 21:39:51,2018-12-28 23:01:16,2018-12-28 21:49:23,K- Spag&Meatball,2.0,1.356944,2,6,18,2,0.0,Friday,True,6,hang_out,0,0,0,130.87,1.0,9.99
360970732,2018-12-28 21:39:51,2018-12-28 23:01:16,2018-12-28 21:49:23,S-Mac&Chz Popprs,2.0,1.356944,2,6,18,2,0.0,Friday,True,6,hang_out,0,0,0,130.87,1.0,5.99
360970732,2018-12-28 21:39:51,2018-12-28 23:01:16,2018-12-28 21:49:23,Maple Nut Salmon,2.0,1.356944,2,6,18,2,0.0,Friday,True,6,hang_out,0,0,0,130.87,1.0,18.99
360970732,2018-12-28 21:39:51,2018-12-28 23:01:16,2018-12-28 22:03:36,S-Moroccan Cigrs,2.0,1.356944,3,6,18,5,0.0,Friday,True,6,hang_out,0,0,0,130.87,1.0,5.99


**Dec 28th (Friday), 21:39 - 23:01, 1.35 hrs, 4 meal steps.**

**Total drinks**: No Drinks

**`Family Occasion`** 

late night, no drinks, K dish



# Table 3

In [26]:
silvester[silvester.index == picked_silvester_tables[2]].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
360903046,2018-12-28 19:25:00,2018-12-28 20:46:29,2018-12-28 19:30:00,CRAB.CAKES*,2.0,1.358056,1,19,20,1,0.0,Friday,True,7,dinner,1,0,0,249.2,1.0,17.95
360903046,2018-12-28 19:25:00,2018-12-28 20:46:29,2018-12-28 19:30:00,PF-CHEESECAKE,2.0,1.358056,1,19,20,1,0.0,Friday,True,7,dinner,1,0,0,249.2,1.0,0.00
360903046,2018-12-28 19:25:00,2018-12-28 20:46:29,2018-12-28 19:30:00,PF-CHEESECAKE,2.0,1.358056,1,19,20,1,0.0,Friday,True,7,dinner,1,0,0,249.2,1.0,0.00
360903046,2018-12-28 19:25:00,2018-12-28 20:46:29,2018-12-28 19:30:00,CHICKEN.CAESAR,2.0,1.358056,1,19,20,1,0.0,Friday,True,7,dinner,0,0,0,249.2,1.0,17.95
360903046,2018-12-28 19:25:00,2018-12-28 20:46:29,2018-12-28 19:30:00,R.BER.CAL.CAB.1z,6.0,1.358056,1,19,20,1,0.0,Friday,True,7,dinner,0,0,0,249.2,1.0,14.50
360903046,2018-12-28 19:25:00,2018-12-28 20:46:29,2018-12-28 19:30:00,GRILL.RIBS,2.0,1.358056,1,19,20,1,0.0,Friday,True,7,dinner,0,0,0,249.2,1.0,31.95
360903046,2018-12-28 19:25:00,2018-12-28 20:46:29,2018-12-28 19:30:00,PF-GARDEN.SALAD,2.0,1.358056,1,19,20,1,0.0,Friday,True,7,dinner,0,0,0,249.2,1.0,0.00
360903046,2018-12-28 19:25:00,2018-12-28 20:46:29,2018-12-28 19:30:00,PF-PAN.SALMON.,2.0,1.358056,1,19,20,1,0.0,Friday,True,7,dinner,0,0,0,249.2,1.0,40.00
360903046,2018-12-28 19:25:00,2018-12-28 20:46:29,2018-12-28 19:30:00,CHKN.COBB.SALAD,2.0,1.358056,1,19,20,1,0.0,Friday,True,7,dinner,0,0,0,249.2,1.0,14.95


**Dec 28th (Friday), 19:25 - 20:46, 1.35 hrs, 1 meal steps.**

**Total drinks**: 1 wine

**`dinner`** 

`late night`, `1 drink` 


